# **Text Preprocessing**


In [1]:
import re
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tokenizers import Tokenizer
from collections import Counter

In [ ]:
def clean_text(text):
    char_map = {
        'ñ': 'n', 'ā': 'a', 'ī': 'i', 'ū': 'u', 'ḳ': 'k', 'ġ': 'g', 'Ḍ': 'D', 'D': 'D', 'ṣ': 's', 'ż': 'z', 'ḷ': 'l',
        'ḥ': 'h', 'ṭ': 't', 'ḍ': 'd', 'ś': 's', 'ṁ': 'm', 'ṣ': 's', 'é': 'e', 'ó': 'o'
    }

    # Replace special characters
    for special_char, normal_char in char_map.items():
        text = text.replace(special_char, normal_char)

    # Remove dots, commas, quotes, and question marks
    text = re.sub(r'[.,"\'?]', '', text)

    # Remove extra spaces but preserve new lines
    text = re.sub(r'[ \t]+', ' ', text).strip()

    return text

# Read CSV file
df = pd.read_csv('Roman-Urdu-Poetry.csv')

# Apply cleaning function to the 'Poetry' column
df['Poetry'] = df['Poetry'].astype(str).apply(clean_text)

# Print cleaned dataframe
print(df.loc[0, 'Poetry'])
# Save the cleaned DataFrame to a new CSV file
df.to_csv('Cleaned_Roman_Urdu_Poetry.csv', index=False)

# Download the file to your PC
# from google.colab import files
# files.download('Cleaned_Roman_Urdu_Poetry.csv')



aankh se duur na ho dil se utar jaega 
vaqt ka kya hai guzarta hai guzar jaega 
itna manus na ho khalvat-e-gham se apni 
tu kabhi khud ko bhi dekhega to Dar jaega 
Dubte Dubte kashti ko uchhala de duun 
main nahin koi to sahil pe utar jaega 
zindagi teri ata hai to ye jaane vaala 
teri bakhshish tiri dahliz pe dhar jaega 
zabt lazim hai magar dukh hai qayamat ka faraz 
zalim ab ke bhi na roega to mar jaega


# With Charater Pair Encoding

In [ ]:
# Character Tokenizer
def char_tokenizer(text):
    vocab = sorted(set(text))
    token_to_id = {char: idx + 1 for idx, char in enumerate(vocab)}  # Reserve 0 for padding
    token_to_id['<PAD>'] = 0
    return token_to_id

# Tokenize the entire dataset
all_text = ' '.join(df['Poetry'].dropna().tolist())
token_to_id = char_tokenizer(all_text)
vocab_size = len(token_to_id)

# Encode Poetry
def encode_poetry(poem):
    return [token_to_id.get(char, 0) for char in poem]  # 0 for unknown characters

poetry_data = df['Poetry'].dropna().tolist()
encoded_poetry = [encode_poetry(poem) for poem in poetry_data]

# Dataset Class
class ShayariDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        return torch.tensor(seq[:-1]), torch.tensor(seq[1:])  # Input-Target pair

# Custom Collate Function for Padding
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=0)
    return inputs_padded, targets_padded

# DataLoader
dataset = ShayariDataset(encoded_poetry)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, collate_fn=collate_fn)

# LSTM Model
class ShayariLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout=0.2):
        super(ShayariLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        output, hidden = self.lstm(x, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size, hidden_dim, num_layers, device):
        return (torch.zeros(num_layers, batch_size, hidden_dim).to(device),
                torch.zeros(num_layers, batch_size, hidden_dim).to(device))

# Model Initialization
embed_dim = 256
hidden_dim = 512
num_layers = 2
dropout = 0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ShayariLSTM(vocab_size, embed_dim, hidden_dim, num_layers, dropout).to(device)

# Training Loop
num_epochs = 100
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, num_epochs + 1):
    model.train()
    hidden = model.init_hidden(batch_size=32, hidden_dim=hidden_dim, num_layers=num_layers, device=device)

    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs}", leave=False)

    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        hidden = tuple(h.detach() for h in hidden)

        output, hidden = model(inputs, hidden)
        output = output.view(-1, output.size(2))
        targets = targets.view(-1)

        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(output, dim=1)
        mask = targets != 0
        correct_predictions = (predicted[mask] == targets[mask]).sum().item()
        total_valid_tokens = mask.sum().item()

        epoch_loss += loss.item()
        epoch_correct += correct_predictions
        epoch_total += total_valid_tokens

        progress_bar.set_postfix({
            'Loss': loss.item(),
            'Accuracy': f"{correct_predictions / total_valid_tokens:.4f}" if total_valid_tokens > 0 else "N/A"
        })

    avg_loss = epoch_loss / len(dataloader)
    epoch_accuracy = epoch_correct / epoch_total if epoch_total > 0 else 0
    print(f"Epoch {epoch}, Average Loss: {avg_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
    if epoch % 10 == 0:
        torch.save(model.state_dict(), f"shayari_lstm_epoch{epoch}.pth")
        torch.save(optimizer.state_dict(), f"optimizer_epoch{epoch}.pth")
        print(f"✅ Model saved at epoch {epoch}")

torch.save(model.state_dict(), "shayari_lstm_final.pth")
torch.save(optimizer.state_dict(), "optimizer_final.pth")
print("✅ Final model saved successfully!")

Epoch 1, Average Loss: 2.5046, Accuracy: 0.3100


Epoch 2, Average Loss: 1.9382, Accuracy: 0.4259


Epoch 3, Average Loss: 1.7655, Accuracy: 0.4681


Epoch 4, Average Loss: 1.6620, Accuracy: 0.4948


Epoch 5, Average Loss: 1.5920, Accuracy: 0.5132


...

Epoch 96, Average Loss: 0.6030, Accuracy: 0.8017


Epoch 97, Average Loss: 0.5976, Accuracy: 0.8034


Epoch 98, Average Loss: 0.5940, Accuracy: 0.8043


Epoch 99, Average Loss: 0.5893, Accuracy: 0.8055


Epoch 100, Average Loss: 0.5860, Accuracy: 0.8069
✅ Model saved at epoch 100
✅ Final model saved successfully!


In [ ]:
# Load Model
model = ShayariLSTM(vocab_size, embed_dim, hidden_dim, num_layers, dropout).to(device)
model.load_state_dict(torch.load("shayari_lstm_final.pth"))  # Load saved weights
model.eval()  # Set model to evaluation mode

# Text Generation Function
def generate_text(start_seq, gen_length=50):
    model.eval()  # Ensure model is in evaluation mode
    tokens = [token_to_id.get(char, 0) for char in start_seq]
    input_seq = torch.tensor(tokens).unsqueeze(0).to(device)
    hidden = model.init_hidden(1, hidden_dim, num_layers, device)

    generated = tokens

    with torch.no_grad():
        for _ in range(gen_length):
            output, hidden = model(input_seq, hidden)
            prob = torch.softmax(output[:, -1, :], dim=-1)
            next_token = torch.argmax(prob, dim=-1).item()

            generated.append(next_token)
            input_seq = torch.tensor([[next_token]]).to(device)

    # Convert IDs back to characters
    id_to_token = {v: k for k, v in token_to_id.items()}
    return ''.join([id_to_token.get(token, '') for token in generated])

# Generate Sample Shayari
sample_shayari = generate_text("tujh pe uthi hain", gen_length=300)
print("\nGenerated Shayari:\n", sample_shayari)



Generated Shayari:
 tujh pe uthi hain sab ke sab khvab bhi ho gaya 
us ki tegh-e-nigah ko abhi tabassum uzr 
yuun to sab kuchh andaz-e-gul-e-tar ka taara 
ki jaise abru-e-asar-e-bahar se vo sun 
main apne aap men sar tha aur bulandi tu 
vasl bhi saaya surashon ka arman ho gaya hasil 
abhi faqirana na main nashsha-e-mai bahut thi 
main 
